<a href="https://colab.research.google.com/github/LuisaPolicarpo/Berlin-Bike-Sharing/blob/main/Berlin_Bike_Sharing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

In [7]:
# --- Connect to CityBikes API ---
url = "http://api.citybik.es/v2/networks/nextbike-berlin"
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()

    # Extract station data
    stations = data['network']['stations']

    # Create a DataFrame
    df_stations = pd.DataFrame(stations)


**EDA**

In [8]:
df_stations['free_bikes'] = df_stations['free_bikes'].fillna(0)
df_stations['empty_slots'] = df_stations['empty_slots'].fillna(0)

In [9]:
df_stations['total_slots'] = df_stations['free_bikes'] + df_stations['empty_slots']
df_stations['occupancy_rate'] = round(df_stations['free_bikes'] / df_stations['total_slots'],2)

In [10]:
df_stations.head()

,id,name,latitude,longitude,timestamp,free_bikes,empty_slots,extra,total_slots,occupancy_rate
0,000c1394c49338dd136955cfa83f0f02,virtuell - U Ullsteinstraße (Staubenrauchbrücke),52.453146,13.384738,2025-04-29T19:57:30.470807+00:00Z,0,6.0,"{'uid': '134099782', 'number': '10209', 'slots...",6.0,0.00
1,000d6738d33fd09713f9ba3d645eb083,virtuell - Olivaer Platz (Nordwest),52.499709,13.313215,2025-04-29T19:57:30.519353+00:00Z,2,2.0,"{'uid': '136246186', 'number': '10445', 'slots...",4.0,0.50
2,00262119a775f009135c009c642de971,Katzbachstraße | BONUS-Station: Return(Rückgab...,52.487904,13.376718,2025-04-29T19:57:30.360811+00:00Z,2,13.0,"{'uid': '3173794', 'number': '1449', 'slots': ...",15.0,0.13
3,0029bd4fba89c502af0c1732591080cd,virtuell - Greifswalder Straße/Lehderstraße,52.547140,13.447640,2025-04-29T19:57:30.597575+00:00Z,0,4.0,"{'uid': '140006670', 'number': '11039', 'slots...",4.0,0.00
4,00455239920d92cdc60461f8b8619fac,virtuell - Linienstraße/Weydingerstraße,52.527005,13.414112,2025-04-29T19:57:30.526221+00:00Z,5,0.0,"{'uid': '138056052', 'number': '10740', 'slots...",5.0,1.00


In [11]:
df_stations.columns

Index(['id', 'name', 'latitude', 'longitude', 'timestamp', 'free_bikes',
       'empty_slots', 'extra', 'total_slots', 'occupancy_rate'],
      dtype='object')

In [12]:
df_stations = df_stations.drop(columns=['id', 'extra'])

In [13]:
df_stations

,name,latitude,longitude,timestamp,free_bikes,empty_slots,total_slots,occupancy_rate
0,virtuell - U Ullsteinstraße (Staubenrauchbrücke),52.453146,13.384738,2025-04-29T19:57:30.470807+00:00Z,0,6.0,6.0,0.00
1,virtuell - Olivaer Platz (Nordwest),52.499709,13.313215,2025-04-29T19:57:30.519353+00:00Z,2,2.0,4.0,0.50
2,Katzbachstraße | BONUS-Station: Return(Rückgab...,52.487904,13.376718,2025-04-29T19:57:30.360811+00:00Z,2,13.0,15.0,0.13
3,virtuell - Greifswalder Straße/Lehderstraße,52.547140,13.447640,2025-04-29T19:57:30.597575+00:00Z,0,4.0,4.0,0.00
4,virtuell - Linienstraße/Weydingerstraße,52.527005,13.414112,2025-04-29T19:57:30.526221+00:00Z,5,0.0,5.0,1.00
...,...,...,...,...,...,...,...,...
4049,virtuell - Schellingstraße (Reichpietschufer),52.505303,13.372947,2025-04-29T19:57:30.412400+00:00Z,1,3.0,4.0,0.25
4050,virtuell - Gerichtstraße/Adolfstraße,52.544916,13.367097,2025-04-29T19:57:30.610279+00:00Z,0,4.0,4.0,0.00
4051,virtuell - Zehlendorf Eiche (Clayallee),52.434834,13.260059,2025-04-29T19:57:30.701909+00:00Z,0,4.0,4.0,0.00
4052,virtuell - Schwedter Straße/Choriner Straße,52.534172,13.408529,2025-04-29T19:57:30.676476+00:00Z,1,3.0,4.0,0.25
